In [0]:
text = ' '
files = os.listdir('./randomsong')
for f in files[:10]:
  with open('./randomsong/' + f) as f:
    text += str(f.read())


text = text.split()
print(text)
words = sorted(list(set(text)))
word_indices = dict((d,i) for i, d in enumerate(words))
indices_word = dict((i,d) for i, d in enumerate(words))

print(len(text), len(words))
print('Vectorization...')

x = np.zeros((len(text),len(words)),dtype=np.bool)
y = np.zeros((len(text),len(words)),dtype=np.bool)
for i in range(len(text)-1):
  x[i,word_indices[text[i    ]]] = 1
  y[i,word_indices[text[i+1]]] = 1

  
print('build model...')
model = Sequential()
model.add(Embedding(len(words), output_dim=256))
model.add(LSTM(128))
model.add(Dense(len(words)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer)


def sample(preds,start):    #start is not used now
  preds = np.asarray(preds).astype('float64')
  #preds = np.log(preds)
  #exp_preds = np.exp(preds)
  #preds = exp_preds / np.sum(exp_preds)

  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)
  

def on_epoch_end(epoch, logs):
  start = indices_word[np.random.randint(0,len(words))]
  sys.stdout.write(start)
  sys.stdout.write(' ')
  for i in range(50):
    x_predict = np.zeros((1,len(words)),dtype=np.bool)
    x_predict[0,word_indices[start]]=1
    prediction = model.predict(x_predict, verbose=0)[0]
    #start = indices_word[np.argmax(prediction)]
    start = indices_word[sample(prediction,start)]
    sys.stdout.write(start)
    sys.stdout.write(' ')
    sys.stdout.flush()
  print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
model.fit(x,y,batch_size=32,epochs=10,callbacks = [print_callback])  